In [ ]:
column set name                 => condition (ADHD, gout)
         [website urls top 10]                 => [drugs]
      all the other columns of the url row => star ratings (0-10) for drug and condition

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()
bw_user = os.environ['bw_user']
bw_password = os.environ['bw_password']
tfn = 'technologies.csv'

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup
import io
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import time
import sys

In [11]:
driver = webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver')
driver.get("http://biglistofwebsites.com/")
# print(driver.title)
assert "Big Lists of Website" in driver.title
urls = []

In [4]:
for i in range(81,0x5B):
  for j in range(75,0x5B):
    search = f"{chr(i)}{chr(j)}AA"
    elem = driver.find_element_by_css_selector("input[name='redirect']")
    elem.clear()
    elem.send_keys(search)
    elem.send_keys(Keys.RETURN)
    #time.sleep(5)
    source = driver.page_source
    assert "No results found." not in driver.page_source
    html = BeautifulSoup(source, 'html.parser')
    urls.extend([tx.text for tx in html.find_all('a', {'rel': 'nofollow'}, {'target': '_blank'})])
    
f = io.open('urls.csv', 'w')
f.write(",".join(urls))
f.close()
print(len(urls))    
driver.close()

1211


In [12]:
class technologies_class:
    def __init__(self, find_dups = False):
        self.data = []
        self.dups = []
        self.find_dups = find_dups
        try:
          f = io.open(tfn, 'r')
          source = f.read()
          f.close()
          for technology in source.split(','):
            self.add(technology)
        except IOError:
            # print('IOError')
            pass

    def add(self, technology):
        technology = technology.strip()      
        try:
            i = self.data.index(technology)
        except ValueError:
            pass
        else:
            if self.find_dups:
                self.dups.append(technology)
            return i
        if technology[0] in ' \t\n':
            print('shit')
        self.data.append(technology)
        return self.data.index(technology)

    def technologies(self):
        for t in self.data:
            if t[0] in ' \t\n':
                print('double shit')
            yield t
    def print_dups(self):
        print(f"count: {len(self.dups)}")
        print(', '.join(self.dups))
    def store(self):
        f = io.open(tfn, 'w')
        f.write(', '.join(self.data))
        f.close()

# technology.add('Google Analytics')
# technology.store()

In [13]:
technology = technologies_class()

In [180]:
f = io.open('urls.csv', 'r')
urls_ = f.read()
f.close()
urls = urls_.split(',')
with_tech_url = 'https://pro.builtwith.com/'
# with_tech_url = 'https://www.wappalyzer.com/'
driver = webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver')
# dir(driver)
wait = WebDriverWait(driver, 10)
driver.get(with_tech_url)
    #elem = driver.find_element_by_css_selector('input[placeholder="Website, Tech, Keyword"]')
    # <a class="text-light" href="//builtwith.com/login">Log In</a>
time.sleep(1)
# elem = driver.find_element_by_css_selector(r'a[href="//builtwith.com/login"]')
# elem.click()
print(driver.title)
WebDriverWait(driver, 10).until(EC.title_contains("Log in"))
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='email']")))
elem.send_keys(bw_user)
# elem.send_keys(Keys.TAB) 
time.sleep(1)
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='password']")))
elem.send_keys(bw_password)
# elem.send_keys(Keys.TAB)  
time.sleep(1)
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='submit']")))
elem.click()
rows = []
for url in urls:
#     if url in ['more info','collapse', 'Privacy Policy', 'Terms of Use' ]:
#         continue
  if url.find('.') > 0:
    # print(url)
    #elem = driver.find_element_by_css_selector('input[placeholder="Website, Tech, Keyword"]')
    elem = driver.find_element_by_css_selector("input[name='q']")
    elem.clear()
    elem.send_keys(url)
    elem.send_keys(Keys.RETURN)
    assert "No results found." not in driver.page_source
    source = driver.page_source
#     f = io.open(f'{url}.html', 'w')
#     f.write(source)
#     f.close()    
    html = BeautifulSoup(source, 'html.parser')
    h2as = []
    for h2 in html.find_all('h2'):
        try:
            h2a = h2.find_next('a', {'class': 'text-dark'}).text
        except:
            pass
        else:
            h2as.append(h2a)
    #print(f"len h2as  {len(h2as)}")
    row = {'url': url}
    for h2a in h2as:
        i = technology.add(h2a)
        if (i % 1000) == 0:
          print(f"i: {i}")
        # assert i < 10, "for debugging"
        row[i] = 1
    rows.append(row)
#     if i > 10:
#         break
df = pd.DataFrame(rows)
print(df.head())
print(df.shape)
df.to_pickle('sites.2.pkl') 
technology.store()
try:
  driver.close()
except WebDriverException:
  print('driver.close failed') 

Log in to BuiltWith
i: 0
i: 0
i: 0
i: 2000
i: 3000
i: 4000
i: 3000
i: 5000
i: 5000
i: 6000
i: 5000
i: 7000
i: 2000
i: 2000
i: 5000
i: 8000
i: 9000
i: 6000
i: 8000
i: 0
i: 9000
i: 0
i: 10000
i: 8000
i: 4000
i: 4000
i: 3000
                   url    0    1    2    3    4    5    6    7    8  ...    \
0         hanatour.com  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...     
1  digwebinterface.com  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...     
2             xfree.hu  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...     
3      iplanetwork.com  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...     
4           sdu.org.cn  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...     

   1137  10979  10980  10981  10982  10983  10984  10985  10986  10987  
0   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
1   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
2   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
3   NaN    NaN   

<a class="text-dark" href="https://trends.builtwith.com/analytics/Google-Analytics">Google Analytics</a>

In [24]:
df = pd.read_pickle('sites.pkl')
df.head()

,url,0,1,2,3,4,5,6,7,8,...,1137,10979,10980,10981,10982,10983,10984,10985,10986,10987
0,hanatour.com,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,digwebinterface.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,xfree.hu,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,iplanetwork.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,sdu.org.cn,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [25]:
df[df.url == 'hanatour.com']

,url,0,1,2,3,4,5,6,7,8,...,1137,10979,10980,10981,10982,10983,10984,10985,10986,10987
0,hanatour.com,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [26]:
import numpy as np
f = open(tfn, 'r')
names_s = f.read()
f.close()
names = np.array(names_s.split(','))
len(names)



10989

In [27]:
d = {}
for i,x in enumerate(names):
  d[i] = x.strip()
df.rename(index=str, columns=d, inplace=True)
df.head()

,url,Google Analytics,Matomo,Facebook Signal,Facebook Pixel,Facebook Conversion Tracking,Google Tag Manager,Korean,Cart Functionality,ASP.NET,...,Piclens Lite,Vid.me,Global Layer,Klasik Framework,Resy,InfinityGrid,moofx,Meteor Slides,Calendly,At.js
0,hanatour.com,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,digwebinterface.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,xfree.hu,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,iplanetwork.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,sdu.org.cn,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [28]:
df.shape

(2659, 10588)

In [17]:
df[df.columns.drop('url')] = df[df.columns.drop('url')].fillna(-1.0).astype(int)

In [18]:
df.head()

,url,0,1,2,3,4,5,6,7,8,...,1137,10979,10980,10981,10982,10983,10984,10985,10986,10987
0,hanatour.com,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,digwebinterface.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,xfree.hu,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,iplanetwork.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,sdu.org.cn,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [29]:
df.to_pickle('sites.pkl')

In [31]:
df = pd.read_pickle('sites.pkl')
df.head()

,url,Google Analytics,Matomo,Facebook Signal,Facebook Pixel,Facebook Conversion Tracking,Google Tag Manager,Korean,Cart Functionality,ASP.NET,...,Piclens Lite,Vid.me,Global Layer,Klasik Framework,Resy,InfinityGrid,moofx,Meteor Slides,Calendly,At.js
0,hanatour.com,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,digwebinterface.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,xfree.hu,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,iplanetwork.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,sdu.org.cn,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [37]:
df.shape

(2659, 10588)

In [34]:
df[(df.url == 'qvae.blogspot.com')].Blogger

2363   -1
Name: Blogger, dtype: int64

In [35]:
df[df.url == 'digwebinterface.com']

,url,Google Analytics,Matomo,Facebook Signal,Facebook Pixel,Facebook Conversion Tracking,Google Tag Manager,Korean,Cart Functionality,ASP.NET,...,Piclens Lite,Vid.me,Global Layer,Klasik Framework,Resy,InfinityGrid,moofx,Meteor Slides,Calendly,At.js
1,digwebinterface.com,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [36]:
def get_columns(key_, exact = False):
  c = []
  if type(key_) is not list:
        key_ = [key_]
  for key in key_:        
    for column in df.columns:
      if exact:
        try:
          assert(column == key),'no match'
          key = key.lower()
        except AssertionError:
          continue
      try:
        index = column.lower().index(key)
      except ValueError:
        pass
      else:
        if column not in c:
          c.append(column)
  return c

phpColumns = get_columns('php')
reduxColumns = get_columns('redux')
reactColumns = get_columns('react')
jqueryColumns = get_columns('jquery')
facebookColumns = get_columns('facebook')
neuralColumns = get_columns('neural')
print(neuralColumns)
wordpressColumns = get_columns('wordpress')
twitterColumns = get_columns('twitter')
angularColumns = get_columns('angular')
vueColumns =  get_columns(['Vue','Vuetify','vuex','Revue'],True)
typescriptColumns = get_columns('typescript')
vueColumns

['Neural Networks']


['Vue', 'vuex']

In [108]:
def phpColumn(row):
  for php in phpColumns:
    if row[php] == 1:
      return 1
  return 0


In [109]:
df['job'] = df.apply(lambda row: phpColumn(row),axis=1)

In [111]:
df.url[df.job == 1].count()

19

In [112]:
df.to_pickle('sites.pkl') 

In [38]:
test_dups = technologies_class(find_dups=True)
test_dups.print_dups()

count: 0



In [39]:
import random
random.seed(123)
ratings = [(t.strip(), random.random() - 0.5) for t in test_dups.technologies() ]
print(f"|{ratings[1][0]}|")

f = io.open('ratings.csv', 'w')
f.write(','.join(f"({r[0].strip()},{r[1]})" for r in ratings))
f.close()

|Matomo|


In [65]:
rating_dict = {}
for r in ratings:
    rating_dict[r[0]] = r[1]

In [135]:
df.shape

(2659, 10617)

In [ ]:

minLen = 30
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

half_mean_height = 0.1

def printChart(c,data):

  df = pd.DataFrame.from_dict(data)
  # print(df.head())
  n = len(data)
  plt.figure(figsize=(max(n/5,5),5))
  data_order =['min_stars', 'low', 'mean_low', 'mean_high', 'high', 'max_stars']
  colors = [None, "b",'g', 'black', "g", "gold"]
#   values = np.array([[d[name] for name in data_order] for d in data])
  order = np.array(data_order)
  ind = np.arange(n)    # the x locations for the groups
  bottom = np.array(df['min_stars'])
  width = 0.4
  name_dict = {'low': 'Low Stars','mean_low': 'Confidence Interval','mean_high': 'Confidence Interval Mean',  'high': None, 'max_stars': 'High Stars'}
  for name,color in zip(data_order[1:],colors[1:]):
    value = np.array(df[name])
    # p1 = plt.bar(ind, value, width, color=color, bottom = bottom, label=name_dict[name], alpha=(1 if name == 'mean_low' else 0.5))
    p1 = plt.bar(ind, value, width, color=color, bottom = bottom, label=name_dict[name]) 
    bottom = value
    # print(f'name: {name} color: {color} value: {value}')
  #plt.bar(ind,np.array(df.mean_high),width, color='black', bottom=np.array(df.mean_low), label = 'Mean')

#   plt.yticks(bottoms+0.4, ["data %d" % (t+1) for t in bottoms])
  plt.legend(loc="best", bbox_to_anchor=(1.0, 1.00))
  plt.ylabel('Stars')
  plt.title(f"Drugs for {c}")
  plt.xticks(ind, np.array(df['drug']), rotation=('vertical' if n > 3 else 'horizontal'))
#   start, end = plt.yaxis
#   plt.yaxis.set_major_locator(ticker.MultipleLocator((end - start) / 10))
  axes = plt.gca()
  axes.set_ylim([0,10])
  plt.yticks(np.arange(0, 11, 1))
#   plt.subplots_adjust(right=0.85)

  plt.show();

phpColumns = get_columns('php')
reduxColumns = get_columns('redux')
reactColumns = get_columns('react')
jqueryColumns = get_columns('jquery')
facebookColumns = get_columns('facebook')
neuralColumns = get_columns('neural')
print(neuralColumns)
wordpressColumns = get_columns('wordpress')
twitterColumns = get_columns('twitter')
angularColumns = get_columns('angular')
vueColumns =  get_columns(['Vue','Vuetify','vuex','Revue'],True)
typescriptColumns = get_columns('typescript')

In [78]:
# Confidence intervals!
# Similar to hypothesis testing, but centered at sample mean
# Better than reporting the "point estimate" (sample mean)
# Why? Because point estimates aren't always perfect

import numpy as np
from scipy import stats

def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
  return (mean, mean - interval, mean + interval)

def report_confidence_interval(confidence_interval):
  """
  Return a string with a pretty report of a confidence interval.
  
  Arguments:
    confidence_interval - tuple of (mean, lower bound, upper bound)
  
  Returns:
    None, but prints to screen the report
  """
  #print('Mean: {}'.format(confidence_interval[0]))
  #print('Lower bound: {}'.format(confidence_interval[1]))
  #print('Upper bound: {}'.format(confidence_interval[2]))
  s = "our mean lies in the interval ]{:.2}, {:.2}[".format(
      confidence_interval[1], confidence_interval[2])
  s = f"our mean {confidence_interval[0]:.2f} lies in the interval {confidence_interval[1]:.2f} - {confidence_interval[2]:.2f}"
  return s

In [96]:
from functools import reduce
df.drop_duplicates('url',inplace=True)
def conjunction(conditions):
    return reduce(np.logical_or, conditions)
for columns in [
        phpColumns, reduxColumns, reactColumns, jqueryColumns, facebookColumns,
        neuralColumns, wordpressColumns, twitterColumns, angularColumns,
        vueColumns, typescriptColumns
]:
  dd = df[conjunction(
    [df[c] == 1 for c in columns]
  )]
  print(len(dd))
  scores = [
      {'idx': idx, 'url': r.url
       ,'score': reduce((lambda s,c: s + (rating_dict[c] if r[c] == 1 else 0)), df.columns[1:], 0)
      }
      for idx,r in dd.iterrows()
  ]
  scores = sorted(scores, key=lambda d: d['score'], reverse=True)[0:10]
  rows = df[df.url.isin([r['url'] for r in scores])]
#   ci = confidence_interval(
#       [(rating_dict[c] if r[c] == 1 else 0) for c in df.columns[1:] ])
  if len(scores) != len(rows):
      print(len(scores),len(rows))
      print(",".join([r['url'] for r in scores]))    
      print(",".join([r.url for _,r in rows.iterrows()]))
#   ci = confidence_interval(
#        [[(rating_dict[c] if r[c] == 1 else 0) for c in df.columns[1:]] for r in rows.iterrows()].flatten())
     

  

15


TypeError: tuple indices must be integers or slices, not str

In [92]:
df.url[df.url == 'jets.in.ua']

1634    jets.in.ua
2194    jets.in.ua
Name: url, dtype: object

In [89]:
urls = ["acaaa.org","lamk.fi","quraanshareef.org","nitrixshop.sk","mixsuplementos.com.br","cartujaesdeporte.es",
 "jets.in.ua","moore-exposure.com","jets.in.ua","pinguinstorello.com","judyguice.com"]
rows = ['cartujaesdeporte.es','lamk.fi','acaaa.org','jets.in.ua','nitrixshop.sk','quraanshareef.org',
        'judyguice.com','moore-exposure.com','mixsuplementos.com.br','pinguinstorello.com']
print(sorted(urls))
print(sorted(rows))

['acaaa.org', 'cartujaesdeporte.es', 'jets.in.ua', 'jets.in.ua', 'judyguice.com', 'lamk.fi', 'mixsuplementos.com.br', 'moore-exposure.com', 'nitrixshop.sk', 'pinguinstorello.com', 'quraanshareef.org']
['acaaa.org', 'cartujaesdeporte.es', 'jets.in.ua', 'judyguice.com', 'lamk.fi', 'mixsuplementos.com.br', 'moore-exposure.com', 'nitrixshop.sk', 'pinguinstorello.com', 'quraanshareef.org']


In [56]:

c1 = df['PHP'] == 1
c2 = df['PHP.NET'] == 1
c3 = df['PHPCart'] == 1
data_filtered = df[conjunction([c1,c2,c3])]
dd = df[conjunction(
    [df[c] == 1 for c in phpColumns]
)]


In [64]:
df.columns[1:]

Index(['Google Analytics', 'Matomo', 'Facebook Signal', 'Facebook Pixel',
       'Facebook Conversion Tracking', 'Google Tag Manager', 'Korean',
       'Cart Functionality', 'ASP.NET', 'Classic ASP',
       ...
       'Piclens Lite', 'Vid.me', 'Global Layer', 'Klasik Framework', 'Resy',
       'InfinityGrid', 'moofx', 'Meteor Slides', 'Calendly', 'At.js'],
      dtype='object', length=10587)

In [ ]:
scores = []
for i,row in enumerate(df.rows):
    scores = []
    for c in df.columns[1:]
        if row[c] != 0:
            scores.append(rating_dict[c])
               
        